In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import matplotlib as mpl
import pyproj

In [2]:
df_agg=pd.read_excel(r"C:\Users\Documents\unal\analisis geoespacial\datos\df_gg.xlsx")
df_agg

,Unnamed: 0,Estacion,pm25,pp_diaria,temperatura,Latitud,Longitud,logpm25
0,0,Barbosa - Torre Social,16.172026,3.605161,20.525628,6.43696,-75.33040,2.783283
1,1,Bello - I.E. Fernando Vélez,18.611428,4.129548,19.154723,6.33755,-75.56780,2.923776
2,2,Caldas - E U Joaquín Aristizabal,20.328367,4.506452,18.172507,6.09308,-75.63776,3.012017
3,3,Copacabana - Ciudadela Educativa La Vida,18.703870,4.129548,20.542402,6.34536,-75.50475,2.928730
4,4,Envigado - E.S.E. Santa Gertrudis,20.007952,5.260258,20.380514,6.16868,-75.58197,2.996130
5,5,Estación Tráfico Centro,31.451636,6.563032,20.995060,6.25256,-75.56958,3.448451
6,6,Itagüí - I.E. Concejo Municipal de Itagüí,19.828446,7.357806,18.851382,6.16850,-75.64436,2.987118
7,7,La Estrella - Hospital,21.028296,9.078452,19.306679,6.15553,-75.64417,3.045869
8,8,"Medellin, Belén - I.E Pedro Justo Berrio",26.117569,4.973484,18.564340,6.23723,-75.61047,3.262608
9,9,Medellín - Santa Elena,12.692992,5.350387,13.277141,6.23636,-75.49847,2.541050


In [4]:
coords_unicas = df_agg.groupby('Estacion')[['Latitud', 'Longitud']].first().values 

In [6]:
import libpysal as ps

w_knn = ps.weights.KNN.from_array(coords_unicas, k=3)

In [12]:
import geopandas as gpd
from libpysal.weights import DistanceBand
gdf = gpd.GeoDataFrame(df_agg, geometry=gpd.points_from_xy(df_agg['Longitud'], df_agg['Latitud']), crs="EPSG:4326")

# Por ejemplo: 100 km de vecindad
w = DistanceBand.from_dataframe(gdf, threshold=100000, binary=True)
w.transform = 'r'

In [14]:
pip install esda

Defaulting to user installation because normal site-packages is not writeable
  Using cached esda-2.7.0-py3-none-any.whl.metadata (2.0 kB)
Using cached esda-2.7.0-py3-none-any.whl (142 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: C:\Users\SKop981040\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [16]:
from esda.moran import Moran
moran_pm25 = Moran(df_agg["pm25"], w, permutations=9999)
print(f"I de Moran: {moran_pm25.I}, p-value: {moran_pm25.p_sim}")


I de Moran: -0.07142857142857147, p-value: 0.0239


C:\Users\SKop981040\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\esda\moran.py:251: RuntimeWarning: invalid value encountered in scalar power
  self.seI_norm = self.VI_norm ** (1 / 2.0)
C:\Users\SKop981040\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\esda\moran.py:264: RuntimeWarning: invalid value encountered in scalar power
  self.seI_rand = VIR ** (1 / 2.0)


In [7]:
from spreg import ML_Lag


# Variables
X = np.array(df_agg[['pp_diaria', 'temperatura']])
y = np.array(df_agg['logpm25']).reshape(-1, 1)

modelo_sar = ML_Lag(y, X, w=w_knn, name_y='logpm25', name_x=['pp_diaria', 'temperatura'])


print(modelo_sar.summary)


REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :     logpm25                Number of Observations:          15
Mean dependent var  :      3.0190                Number of Variables   :           4
S.D. dependent var  :      0.2029                Degrees of Freedom    :          11
Pseudo R-squared    :      0.6248
Spatial Pseudo R-squared:  0.3882
Log likelihood      :      9.7023
Sigma-square ML     :      0.0146                Akaike info criterion :     -11.405
S.E of regression   :      0.1207                Schwarz criterion     :      -8.572

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
---------------------------------------------------------------

acá podemos observar que el modelo SAR detecta dependencia espacial a diferencia del Moran , con la misma matriz , esto puede pasar debido a que las estaciones están muy dispersas  y moran puede perder poder estadístico y la dependencia puede ser más local que global. Las vecinas afectan a las estaciones actuales 

In [9]:
from spreg import ML_Error

# Variables explicativas
X = df_agg[['pp_diaria', 'temperatura']].values

# Variable dependiente
y = df_agg['logpm25'].values.reshape(-1, 1)


modelo_sem = ML_Error(y, X, w=w_knn, name_y='logpm25', name_x=['pp_diaria', 'temperatura'])


print(modelo_sem.summary)


REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ML SPATIAL ERROR (METHOD = full)
---------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :     logpm25                Number of Observations:          15
Mean dependent var  :      3.0190                Number of Variables   :           3
S.D. dependent var  :      0.2029                Degrees of Freedom    :          12
Pseudo R-squared    :      0.3943
Log likelihood      :      9.0420
Sigma-square ML     :      0.0161                Akaike info criterion :     -12.084
S.E of regression   :      0.1269                Schwarz criterion     :      -9.960

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         1.47243    

C:\Users\SKop981040\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\spreg\ml_error.py:184: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  res = minimize_scalar(
C:\Users\SKop981040\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\spreg\ml_error.py:563: RuntimeWarning: invalid value encountered in log
  jacob = np.log(np.linalg.det(a))


estos resultados quieren decir que mis  variables  o mi modelo no está contemplando alguna variable, debido a que mi lambda presenta un error significativo.